In [12]:
##package needed to do the exercise
import pandas as pd #package for reading and using dataset
import numpy as np #package needed for inserting value in dataset
from sklearn.metrics import roc_auc_score #needed for determing AUC score
import time as t
begin = t.perf_counter()

In [13]:
df=pd.read_csv('C:/Users/s151675/Documents/Jaar_master_2/Kwartiel 3,4/Master project/Data aangepast/data_for_prediction/data+prediction_final/data_complete_final.csv', delimiter = ';') #read csv file

In [14]:
df #visualize dataset

,Case ID,Activity,Complete Timestamp,Variant,Variant index,Specialismecode,Age,Gender,Triage color,Transport,...,Labaanvraag_SEH,Rad_to_ext,Consult_SEH,Opname,Bigger4,ArrivelDate,ArrivelTime,DepartDate,DepartTime,Unnamed: 24
0,155011,Medicatieverificatie SEH,2019-01-01 00:35:00.000,Variant 2252,2252,CHI,26,Man,Geel,Ambulance,...,1,0,1,1,0,2019-01-01 00:00:00.000,"0,024305556",2019-01-01 00:00:00.000,"0,139583333","0,115277778"
1,155013,Radiologie,2019-01-01 00:37:00.000,Variant 2253,2253,NEU,73,Man,Oranje,Ambulance,...,0,0,0,1,1,2019-01-01 00:00:00.000,"0,025694444",2019-01-01 00:00:00.000,"0,227083333","0,201388889"
2,155020,Medicatieverificatie SEH,2019-01-01 01:29:00.000,Variant 4,4,CHI,32,Man,Groen,Eigen vervoer,...,0,0,0,0,0,2019-01-01 00:00:00.000,"0,061805556",2019-01-01 00:00:00.000,"0,086111111","0,024305556"
3,155021,Medicatieverificatie SEH,2019-01-01 01:26:00.000,Variant 2254,2254,INT,34,Man,Oranje,Ambulance,...,1,0,0,1,0,2019-01-01 00:00:00.000,"0,059722222",2019-01-01 00:00:00.000,"0,202083333","0,142361111"
4,155027,Medicatieverificatie SEH,2019-01-01 01:54:00.000,Variant 1252,1252,CAR,21,Vrouw,Geel,Eigen vervoer,...,1,0,0,0,0,2019-01-01 00:00:00.000,"0,079166667",2019-01-01 00:00:00.000,"0,225694444","0,146527778"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50078,243220,Pijnscore herhalen,2020-08-02 21:36:00.000,Variant 455,455,CHI,60,Vrouw,Groen,Eigen vervoer,...,0,0,0,0,0,2020-08-02 00:00:00.000,"0,85625",2020-08-02 00:00:00.000,"0,909027778","0,052777778"
50079,243221,Pijnscore herhalen,2020-08-02 21:36:00.000,Variant 36877,36877,CHI,27,Man,Geel,Eigen vervoer,...,1,1,0,0,0,2020-08-02 00:00:00.000,"0,895833333",2020-08-02 00:00:00.000,"0,994444444","0,098611111"
50080,243223,Beleidsorder SEH,2020-08-02 23:20:00.000,Variant 1587,1587,NEU,77,Vrouw,Geel,Ambulance,...,1,0,0,0,0,2020-08-02 00:00:00.000,"0,972222222",2020-08-03 00:00:00.000,"0,102083333","0,129861111"
50081,243224,Beleidsorder SEH-VPL,2020-08-02 22:48:00.000,Variant 36878,36878,CAR,63,Vrouw,Geel,Eigen vervoer,...,1,0,0,0,0,2020-08-02 00:00:00.000,"0,941666667",2020-08-03 00:00:00.000,"0,045833333","0,104166667"


In [15]:
##packages needed for using decision tree classifier and splitting dataset
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [16]:
#Considered one-hot-encoding in our data, see report.
spec_dummies = pd.get_dummies(df.Specialismecode) #OneHotencoder used for column job
transport_dummies = pd.get_dummies(df.Transport) #OneHotencoder used for column marital
spec_names = spec_dummies.columns
transport_names = transport_dummies.columns
##replace values from categorical to nummerical 
df = df.replace('Vrouw', 0)
df = df.replace('Man', 1)
##replace values from categorical to nummerical (with order)
df = df.replace('Blauw',0) 
df = df.replace('Groen', 1)
df = df.replace('Geel', 2)
df = df.replace('Oranje', 3)
df = df.replace('Rood', 4)

df = pd.concat([df.iloc[:,0],spec_dummies,df.iloc[:,6:9], transport_dummies, df.iloc[:,11], df.iloc[:,14:20]], axis=1) #Dataset with new numerical columns(job and marital)
df



,Case ID,ANE,AVG,CAR,CHI,CTC,DER,GYN,HEM,ICA,...,Anders,Eigen vervoer,Trauma helikopter,History_bin,Presentation_time,Labaanvraag_SEH,Rad_to_ext,Consult_SEH,Opname,Bigger4
0,155011,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,1,1,0
1,155013,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,155020,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,155021,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
4,155027,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50078,243220,0,0,0,1,0,0,0,0,0,...,0,1,0,1,8,0,0,0,0,0
50079,243221,0,0,0,1,0,0,0,0,0,...,0,1,0,1,8,1,1,0,0,0
50080,243223,0,0,0,0,0,0,0,0,0,...,0,0,0,0,9,1,0,0,0,0
50081,243224,0,0,1,0,0,0,0,0,0,...,0,1,0,0,9,1,0,0,0,0


In [17]:
##define input variables(X) and output variable(Y)
feature_cols2 = df.columns
feature_cols = feature_cols2[0:df.shape[1]-5]
feature_cols_check = feature_cols2[1:df.shape[1]-1]
feature_cols_x = feature_cols2[1:df.shape[1]-5] # all columns excluding 'y' and case_ID
feature_cols_y = feature_cols2[df.shape[1]-5:df.shape[1]] 
# feature_cols = ['to_extern', 'INT', 'CHI', 'Age', 'registering_time', 'MDL', 'ORT', 'History_bin', 'CAR', 'Triage color', 'NEU','Lab']#take relevant features to speed up process
X = df[feature_cols] #input variables
X_ana = df[feature_cols_x]
X_ana_big = df[feature_cols_check]
y = df[feature_cols_y]# Target variable
print(X)
print(y)


       Case ID  ANE  AVG  CAR  CHI  CTC  DER  GYN  HEM  ICA  ...  Gender  \
0       155011    0    0    0    1    0    0    0    0    0  ...       1   
1       155013    0    0    0    0    0    0    0    0    0  ...       1   
2       155020    0    0    0    1    0    0    0    0    0  ...       1   
3       155021    0    0    0    0    0    0    0    0    0  ...       1   
4       155027    0    0    1    0    0    0    0    0    0  ...       0   
...        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...     ...   
50078   243220    0    0    0    1    0    0    0    0    0  ...       0   
50079   243221    0    0    0    1    0    0    0    0    0  ...       1   
50080   243223    0    0    0    0    0    0    0    0    0  ...       0   
50081   243224    0    0    1    0    0    0    0    0    0  ...       0   
50082   243225    0    0    0    0    0    0    0    0    0  ...       1   

       Triage color  Ambulance  Ambulance (Inzet heli-MMT)  \
0                 2      

In [18]:
# Split dataset into training set and test set
X_train_t, X_test_t, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # divide dataset into 70% training and 30% test
X_train = X_train_t.iloc[:,1:40]
X_test = X_test_t.iloc[:,1:40]
print(X_train)
y_train_lab = y_train.iloc[:,0]
y_train_rad_to_ext = y_train.iloc[:,1]
y_train_consult = y_train.iloc[:,2]
y_train_opname = y_train.iloc[:,3]
y_test_lab = y_test.iloc[:,0]
y_test_rad_to_ext = y_test.iloc[:,1]
y_test_consult = y_test.iloc[:,2]
y_test_opname = y_test.iloc[:,3]
X_number = X_test_t.iloc[:,0] #take caseID from test set to compare with actual value in later phase
###dataset needed for making predictions with already predicted values
X_train_big = pd.concat([X_train_t.iloc[:,1:40],y_train.iloc[:,0:4]], axis=1)
##choose between these test_data set, where perf is with actual values and other is with predicted values
X_test_big = X_test #make this out of prediction
# X_test_big_perf = pd.concat([X_test_t.iloc[:,1:40],y_test.iloc[:,0:4]], axis=1) #to compare with existing values of columns
y_train_big = y_train.iloc[:,-1]
y_test_big = y_test.iloc[:,-1]

       ANE  AVG  CAR  CHI  CTC  DER  GYN  HEM  ICA  INT  ...  Gender  \
13254    0    0    1    0    0    0    0    0    0    0  ...       0   
22098    0    0    0    1    0    0    0    0    0    0  ...       1   
29082    0    0    0    0    0    0    0    0    0    1  ...       1   
27661    0    0    0    1    0    0    0    0    0    0  ...       1   
37323    0    0    0    0    0    0    0    0    0    0  ...       1   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...     ...   
50057    0    0    0    1    0    0    0    0    0    0  ...       1   
32511    0    0    0    0    0    0    0    0    0    0  ...       1   
5192     0    0    0    1    0    0    0    0    0    0  ...       0   
12172    0    0    0    0    0    0    0    0    0    0  ...       1   
33003    0    0    0    0    0    0    0    0    0    0  ...       0   

       Triage color  Ambulance  Ambulance (Inzet heli-MMT)  \
13254             1          0                           0   
22098      

In [19]:
#Function made to measure the confusing matrix
def perf_measure(y_actual, y_hat):
    ## start with zero values
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    count = 0 
    for i in range(len(y_hat)): #for all values in the column y_test
        count += 1 #count the total number of rows
        if y_actual[i]==1 and y_hat[i]==1:#True positive
            TP += 1
        if y_hat[i]==1 and y_actual[i]==0:#False positive
            FP += 1
        if y_actual[i]==0 and y_hat[i]==0:#True negative
            TN += 1
        if y_hat[i]==0 and y_actual[i]==1:#False negative
            FN += 1

    return(TP/count, FP/count, TN/count, FN/count, count)#return all relevant values in fractions

In [20]:
def RandomForest(Number_leaves, X_train, y_train, X_test):
    clf = RandomForestClassifier(n_estimators = 100, min_samples_leaf=Number_leaves,class_weight = "balanced") #define decision tree with selected variable
    clf = clf.fit(X_train,y_train) #train the decision tree on training set
    y_pred = clf.predict(X_test) #predict the values of X_test
    
    return(clf, y_pred)


In [21]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [22]:
def predict_value(y_train, y_test, X_train, X_test, X_ana, feature_cols2, negative):
    ##used for determing best values
    max_score = 0
    max_AUC = 0
    best_leaves = 0
    y_test = np.array(y_test)
    clf_final, y_pred = RandomForest(2, X_train, y_train, X_test)
    ##loop over values from 2 to 100 for selecting best parameter
    for i in range(2,200):
        clf, y_pred = RandomForest(i,X_train, y_train, X_test)
    #     AUC_score = roc_auc_score(y_test,y_pred)
    #     print(accuracy_score(y_test_lab, y_pred))
        TPRate,FPRate,TNRate, FNRate, count = perf_measure(y_test, y_pred) #call confusing matrix function to determine these values in matrix
        if(negative==False):
            accuracy = (TPRate+TNRate)/(TPRate+TNRate+FPRate+FNRate)
            print("accuracy :", accuracy," for num of leaves =", i)
        else:
            accuracy = TNRate/(TNRate+FNRate)
            print("Negative prediction value :", accuracy," for num of leaves =", i)
        ##function used for updating best score if one score exceed previously best scores
        
        if accuracy>=max_score:
            max_score = accuracy
            best_leaves = i
            clf_final = clf
    
    clf, y_pred = RandomForest(best_leaves,X_train, y_train, X_test) #Best decision tree based on highest accuracy 
    TPRate,FPRate,TNRate, FNRate, count = perf_measure(y_test, y_pred) #call confusing matrix function to determine these values in matrix
    p = (((TNRate+FPRate)*count)/count*((TNRate+FNRate)*count)/count) #measure p value for best model based on recall & AUC score
    kappa_score = (metrics.accuracy_score(y_test, y_pred)-p)/(1-p) #measure Kappa score for best model based on recall & AUC score
    AUC_score = roc_auc_score(y_test,y_pred) #measure AUC score for best model based on recall & AUC score
    ##measure Accuracy score for best model based on Recall & AUC score
    accuracy = (TPRate+TNRate)/(TPRate+TNRate+FPRate+FNRate)

    ###feature importance start
    forest = ExtraTreesClassifier(n_estimators=100,min_samples_leaf=best_leaves, class_weight = "balanced")
    forest = forest.fit(X_train,y_train)

    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
    indices = np.argsort(importances)[::-1]
    ##print all scores for best model based on recall & AUC score
    print("best number of leaves = ", best_leaves, " with score = ", max_score, " and accuracy = ", accuracy)
    print("kappa score =", kappa_score)
    print("AUC score =", AUC_score)
    print(confusion_matrix(y_test,y_pred))
    # Print the feature ranking
    print("Feature ranking:")
    for f in range(X_ana.shape[1]):
        print("%d. %s (%f)" % (f + 1, feature_cols2[indices[f]], importances[indices[f]]))
    return(clf_final)


In [23]:
forest_lab = predict_value(y_train_lab, y_test_lab, X_train, X_test, X_ana, feature_cols_x, False)
forest_rad_to_ext = predict_value(y_train_rad_to_ext, y_test_rad_to_ext, X_train, X_test, X_ana, feature_cols_x, False)
forest_consult = predict_value(y_train_consult, y_test_consult, X_train, X_test, X_ana, feature_cols_x, False)
forest_opname = predict_value(y_train_opname, y_test_opname, X_train, X_test, X_ana, feature_cols_x, False)

lab_prediction = forest_lab.predict(X_test)
rad_to_ext_prediction = forest_rad_to_ext.predict(X_test)
consult_prediction = forest_consult.predict(X_test)
opname_prediction = forest_opname.predict(X_test)
X_test_big.insert(36,"labaanvraag", lab_prediction)
X_test_big.insert(37,"rad_to_ext", rad_to_ext_prediction)
X_test_big.insert(38,"consult", consult_prediction)
X_test_big.insert(39,"opname", opname_prediction)
forest_big4 = predict_value(y_train_big, y_test_big, X_train_big, X_test_big, X_ana_big, feature_cols_check, True) ##have to change X_train last features
big4_prediction = forest_big4.predict(X_test_big)
X_test_big.insert(40,"bigger than 4", big4_prediction)
eind = t.perf_counter()
print("number of seconds =", eind-begin)
dataframeafter = pd.concat([X_number,X_test_big.iloc[:,0:43]],axis=1) ##add bigger4 prediction
dataframeafter.to_csv('C:/Users/s151675/Documents/Jaar_master_2/Kwartiel 3,4/Master project/Data aangepast/Data_for_prediction/final_testgroup_prediction.csv')

accuracy : 0.8011314475873544  for num of leaves = 2
accuracy : 0.7979367720465891  for num of leaves = 3
accuracy : 0.797603993344426  for num of leaves = 4
accuracy : 0.7971381031613977  for num of leaves = 5
accuracy : 0.7934109816971714  for num of leaves = 6
accuracy : 0.7902828618968386  for num of leaves = 7
accuracy : 0.7914808652246257  for num of leaves = 8
accuracy : 0.789351081530782  for num of leaves = 9
accuracy : 0.7899500831946755  for num of leaves = 10
accuracy : 0.7854908485856905  for num of leaves = 11
accuracy : 0.7877537437603994  for num of leaves = 12
accuracy : 0.7860232945091514  for num of leaves = 13
accuracy : 0.786955074875208  for num of leaves = 14
accuracy : 0.7843594009983361  for num of leaves = 15
accuracy : 0.7855574043261231  for num of leaves = 16
accuracy : 0.7844925124792014  for num of leaves = 17
accuracy : 0.7852911813643927  for num of leaves = 18
accuracy : 0.7852911813643927  for num of leaves = 19
accuracy : 0.7816971713810317  for num 

accuracy : 0.763793677204659  for num of leaves = 154
accuracy : 0.7677870216306156  for num of leaves = 155
accuracy : 0.7598668885191348  for num of leaves = 156
accuracy : 0.7717138103161397  for num of leaves = 157
accuracy : 0.7637271214642263  for num of leaves = 158
accuracy : 0.7679866888519136  for num of leaves = 159
accuracy : 0.7693843594009984  for num of leaves = 160
accuracy : 0.7681863560732114  for num of leaves = 161
accuracy : 0.7714475873544093  for num of leaves = 162
accuracy : 0.763594009983361  for num of leaves = 163
accuracy : 0.7568718801996672  for num of leaves = 164
accuracy : 0.7673211314475874  for num of leaves = 165
accuracy : 0.7633277870216306  for num of leaves = 166
accuracy : 0.7631946755407654  for num of leaves = 167
accuracy : 0.7634608985024959  for num of leaves = 168
accuracy : 0.7622628951747088  for num of leaves = 169
accuracy : 0.7664559068219634  for num of leaves = 170
accuracy : 0.7705158069883528  for num of leaves = 171
accuracy : 0

accuracy : 0.6639600665557405  for num of leaves = 89
accuracy : 0.6636272878535774  for num of leaves = 90
accuracy : 0.665757071547421  for num of leaves = 91
accuracy : 0.6652911813643927  for num of leaves = 92
accuracy : 0.6626955074875208  for num of leaves = 93
accuracy : 0.6607653910149751  for num of leaves = 94
accuracy : 0.6629617304492512  for num of leaves = 95
accuracy : 0.6641597337770383  for num of leaves = 96
accuracy : 0.6634276206322796  for num of leaves = 97
accuracy : 0.6591680532445923  for num of leaves = 98
accuracy : 0.6650915141430949  for num of leaves = 99
accuracy : 0.6615640599001664  for num of leaves = 100
accuracy : 0.664891846921797  for num of leaves = 101
accuracy : 0.663161397670549  for num of leaves = 102
accuracy : 0.6651580698835275  for num of leaves = 103
accuracy : 0.6625623960066556  for num of leaves = 104
accuracy : 0.6636272878535774  for num of leaves = 105
accuracy : 0.6606988352745424  for num of leaves = 106
accuracy : 0.66362728785

accuracy : 0.6575041597337771  for num of leaves = 23
accuracy : 0.6523793677204659  for num of leaves = 24
accuracy : 0.6560399334442596  for num of leaves = 25
accuracy : 0.647720465890183  for num of leaves = 26
accuracy : 0.6465890183028287  for num of leaves = 27
accuracy : 0.6505158069883528  for num of leaves = 28
accuracy : 0.6572379367720466  for num of leaves = 29
accuracy : 0.6425956738768719  for num of leaves = 30
accuracy : 0.6561064891846922  for num of leaves = 31
accuracy : 0.651846921797005  for num of leaves = 32
accuracy : 0.6438602329450915  for num of leaves = 33
accuracy : 0.6417970049916806  for num of leaves = 34
accuracy : 0.6425956738768719  for num of leaves = 35
accuracy : 0.6448585690515807  for num of leaves = 36
accuracy : 0.6424625623960067  for num of leaves = 37
accuracy : 0.6433943427620632  for num of leaves = 38
accuracy : 0.6451913477537438  for num of leaves = 39
accuracy : 0.6451913477537438  for num of leaves = 40
accuracy : 0.6412645590682197 

accuracy : 0.6120465890183029  for num of leaves = 174
accuracy : 0.6203660565723794  for num of leaves = 175
accuracy : 0.6191014975041598  for num of leaves = 176
accuracy : 0.6181031613976705  for num of leaves = 177
accuracy : 0.6149084858569052  for num of leaves = 178
accuracy : 0.6277537437603994  for num of leaves = 179
accuracy : 0.615108153078203  for num of leaves = 180
accuracy : 0.6184359400998336  for num of leaves = 181
accuracy : 0.618369384359401  for num of leaves = 182
accuracy : 0.6190349417637272  for num of leaves = 183
accuracy : 0.6193011647254576  for num of leaves = 184
accuracy : 0.615108153078203  for num of leaves = 185
accuracy : 0.6116472545757072  for num of leaves = 186
accuracy : 0.6176372712146423  for num of leaves = 187
accuracy : 0.6132445923460899  for num of leaves = 188
accuracy : 0.621630615640599  for num of leaves = 189
accuracy : 0.616838602329451  for num of leaves = 190
accuracy : 0.6170382695507488  for num of leaves = 191
accuracy : 0.62

accuracy : 0.6809317803660566  for num of leaves = 110
accuracy : 0.6769384359400998  for num of leaves = 111
accuracy : 0.6833943427620632  for num of leaves = 112
accuracy : 0.6754742096505824  for num of leaves = 113
accuracy : 0.6789351081530782  for num of leaves = 114
accuracy : 0.6812645590682196  for num of leaves = 115
accuracy : 0.6808652246256239  for num of leaves = 116
accuracy : 0.6821963394342763  for num of leaves = 117
accuracy : 0.6837936772046589  for num of leaves = 118
accuracy : 0.6847920133111481  for num of leaves = 119
accuracy : 0.6788685524126457  for num of leaves = 120
accuracy : 0.6817304492512479  for num of leaves = 121
accuracy : 0.68  for num of leaves = 122
accuracy : 0.6814642262895175  for num of leaves = 123
accuracy : 0.6847254575707156  for num of leaves = 124
accuracy : 0.6787354409317804  for num of leaves = 125
accuracy : 0.6835274542429285  for num of leaves = 126
accuracy : 0.6794675540765391  for num of leaves = 127
accuracy : 0.67820299500

Negative prediction value : 0.7937426054949388  for num of leaves = 35
Negative prediction value : 0.7928664072632944  for num of leaves = 36
Negative prediction value : 0.7922599016308568  for num of leaves = 37
Negative prediction value : 0.7931851072227339  for num of leaves = 38
Negative prediction value : 0.7942864596921472  for num of leaves = 39
Negative prediction value : 0.7922604261995032  for num of leaves = 40
Negative prediction value : 0.7936279008784581  for num of leaves = 41
Negative prediction value : 0.7914507772020725  for num of leaves = 42
Negative prediction value : 0.7921943654691134  for num of leaves = 43
Negative prediction value : 0.7930989583333333  for num of leaves = 44
Negative prediction value : 0.7927207021166752  for num of leaves = 45
Negative prediction value : 0.7935929157442374  for num of leaves = 46
Negative prediction value : 0.7930584551148226  for num of leaves = 47
Negative prediction value : 0.7932159165035878  for num of leaves = 48
Negati

Negative prediction value : 0.7939032471835653  for num of leaves = 150
Negative prediction value : 0.7928608650338719  for num of leaves = 151
Negative prediction value : 0.7946641890098222  for num of leaves = 152
Negative prediction value : 0.7938184663536777  for num of leaves = 153
Negative prediction value : 0.7932968179447053  for num of leaves = 154
Negative prediction value : 0.7954755920095251  for num of leaves = 155
Negative prediction value : 0.7927263144024246  for num of leaves = 156
Negative prediction value : 0.7973708670827248  for num of leaves = 157
Negative prediction value : 0.791580310880829  for num of leaves = 158
Negative prediction value : 0.7947800741918388  for num of leaves = 159
Negative prediction value : 0.7960543854972008  for num of leaves = 160
Negative prediction value : 0.7943763326226013  for num of leaves = 161
Negative prediction value : 0.7962987442167878  for num of leaves = 162
Negative prediction value : 0.7911105351820656  for num of leaves